# Capture The Flag Shitty Add-On

A (partial so far) solution by [Blake Burkhart](https://bburky.com)

## Setup

I tested this solution with a Raspberry Pi 2 directly connected to a Digispark clone (an ATtiny85 board). The Digispark was powered from the Pi's 3.3v pin. The [`
ctf-firmware-1.0.0.hex`](https://github.com/urish/ctf-shittyaddon/releases/tag/1.0.0) file was flashed to the Digispark.

The Pi must have I²C and SPI enabled in `/boot/config.txt`:

    dtparam=i2c_arm=on,i2c_arm_baudrate=10000
    dtparam=spi=on
  
I set the I²C baud rate to 10kHz to avoid errors. I think this may be due the fact that Raspberry Pis [don't support clock stretching](http://www.advamation.com/knowhow/raspberrypi/rpi-i2c-bug.html) but the [attiny85 and the TinyWire library may allow it](https://github.com/lucullusTheOnly/TinyWire/issues/1). In other words, the slow ATtiny85 may not send all the data fast enough with the Pi's default 100kHz baud rate, set it to something slow like 10kHz. 40kHz was working, but then it quit working some reason.

Also if the AVR ISP is connected to the Pi's SPI pins, the Digispark can be flashed over directly from the Pi with `avrdude`'s linuxspi protocol:

    avrdude -v -pattiny85 -clinuxspi -b14400 -P /dev/spidev0.0 -U flash:w:ctf-firmware-1.0.0.hex 
    avrdude -v -pattiny85 -clinuxspi -b14400 -P /dev/spidev0.0 -Uefuse:w:0xfe:m -Uhfuse:w:0xdf:m -Ulfuse:w:0xe2:m
    
Don't connect the Pi to the Digispark's I²C and SPI at the same time, they share pins and things will go wrong.

# shittyaddon I²C read/write

On an I²C read two bytes are returned: the lower byte of the 16-bit target address, and the byte at the adddress.

When writing, the first byte is used to set the target address. The upper byte is always set to zero. The remainder of the bytes are written in to memory sequentially at the target pointer.

A write with zero data bytes may be performed before a read to set the target address of the read.

Define a pair of `i2c_write()` and `i2c_read()` functions. These functions also track the full 16-byte target address in the `current_target` variable.

In [1]:
from smbus2 import SMBus, i2c_msg
from io import BytesIO

CTF_I2C_ADDR = 0x23

current_target = None

def i2c_write(target, data=[], address=CTF_I2C_ADDR):
    global current_target
    with SMBus(1) as bus:
        msg = i2c_msg.write(address, [target, *data])
        bus.i2c_rdwr(msg)
    current_target = target + len(data)

def i2c_read(target=None, count=1, address=CTF_I2C_ADDR):
    global current_target
    with BytesIO() as f:
        with SMBus(1) as bus:
            # If provided, change target by writing the target address, but no data bytes
            if target is not None:
                msg = i2c_msg.write(address, [target])
                bus.i2c_rdwr(msg)
                current_target = target

            for i in range(count):
                msg = i2c_msg.read(address, 2)
                bus.i2c_rdwr(msg)
                target, data = msg
                f.write(bytes([data]))
                
                if current_target is not None:
                    assert(target == current_target & 0xff)
                current_target += 1
    
        f.seek(0)
        return f.read()

Some quick testing. Manually set the target to 0x00 and read one byte:

In [2]:
i2c_write(0x00)
i2c_read()

b'"'

Read 4 bytes starting at 0x00:

In [3]:
i2c_read(target=0x00, count=4)

b'"\x00\x04@'

Write 6 bytes starting at 0x00:

In [4]:
# i2c_write(0x00, b'FOOBAR')

# Set LED via I2C

In [5]:
# Values appear in datasheets, I found __SFR_OFFSET in avr-libc
SFR_OFFSET = 0x20
PORTB_ADDR = 0x18 + SFR_OFFSET
DDRB_ADDR = 0x17 + SFR_OFFSET

In [6]:
# Read the current value of DDRB
ddrb, = i2c_read(DDRB_ADDR)

# Configure LED as an output pin
# Enable DDB1 for the LED on a digispark 
# I think this is the same pin on the shittyaddon? It uses "pin 6 of the ATtiny (a.k.a PB1)"
ddrb |= 1 << 1

# Set DDRB on the board
i2c_write(DDRB_ADDR, [ddrb])

# Set all output bits in PORTB to turn on the LED
i2c_write(PORTB_ADDR, [0xff])

# Dump flash and find the flag

Dump the entire flash (plus some, it loops if you read too much?) and search for the flag with a regex on the bytes.

In [7]:
import re

dump = i2c_read(target=0x00, count=0x10000)

match = re.search(b'\$FLAG:.*?(?=\x00)', dump)
print(match.group(0).decode('ascii'))

$FLAG:you_found_it!!! :-)


In [8]:
start = match.start()
flag_start = start + len(b'$FLAG:')
end = match.end()

print(hex(flag_start))

0x8024


# TODO: Blinking rootkit

# TODO: Replace the flag

These will require finally figuring out whatever is acually going on in `loop()`.